<a href="https://colab.research.google.com/github/julrods/cyber-bullying-detector/blob/main/4_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# Environment

## Libraries

In [ ]:
!pip install transformers

In [ ]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from transformers import TFBertModel, BertConfig, TFBertForSequenceClassification, BertTokenizer
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from IPython.display import display
import re
import unicodedata
import nltk
from nltk.corpus import stopwords
import itertools
from tqdm import tqdm
import pickle
from sklearn.utils import shuffle
import tweepy
import json
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
filepath = '/content/gdrive/MyDrive/Cyber-bullying-project/keys/twitter_credentials.json'

In [ ]:
with open(filepath) as data_file:
    credentials = json.load(data_file)
auth = tweepy.OAuthHandler(credentials['api_key'], credentials['api_secret_key'])
auth.set_access_token(credentials['access_token'], credentials['access_token_secret'])
api = tweepy.API(auth)

## Functions

In [ ]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def clean_stopwords_shortwords(w):
    stopwords_list=stopwords.words('english')
    words = w.split() 
    clean_words = [word for word in words if (word not in stopwords_list) and len(word) > 2]
    return " ".join(clean_words) 

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"([?.!,¿])", r" ", w)
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    w=clean_stopwords_shortwords(w)
    w=re.sub(r'@\w+', '',w)
    return w

# Model

In [ ]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5,
                                   epsilon=1e-08)

model.compile(loss = loss, optimizer = optimizer, metrics = [metric])
model.load_weights('/content/gdrive/MyDrive/Cyber-bullying-project/models/aggression_model_1epoch.h5')

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Twitter API

## Tweet extraction

In [ ]:
user = 'DonaldJTrumpJr' #request.form['message']

In [ ]:
tweet_dict = [{'tweet': tweet.full_text,
              'created_at': tweet.created_at, 
              'username': user,
              'headshot_url': tweet.user.profile_image_url,
              'url': f'https://twitter.com/user/status/{tweet.id}'
               } for tweet in tweepy.Cursor(api.user_timeline,
                                            screen_name=user,
                                            exclude_replies=True,
                                            include_rts=False,
                                            tweet_mode = "extended").items(100)]

In [ ]:
text_list = [tweet['tweet'] for tweet in tweet_dict]

In [ ]:
tweets_clean = list(map(preprocess_sentence, text_list))

In [ ]:
tweets_final = list(map(lambda x: x.split('http')[0] if 'http' in x else x, tweets_clean))

In [ ]:
input_ids = []
attention_masks = []
for tweet in tweets_final:
    bert_inp = bert_tokenizer.encode_plus(tweet,
                                          add_special_tokens = True,
                                          max_length = 100,
                                          truncation = True,
                                          padding = 'max_length',
                                          return_attention_mask = True)
    # Append every sentence vector to a list
    input_ids.append(bert_inp['input_ids'])
    attention_masks.append(bert_inp['attention_mask'])

# Convert the lists to arrays so that we can input them into the model
input_ids = np.asarray(input_ids)
attention_masks = np.array(attention_masks)

In [ ]:
preds = model.predict([input_ids, attention_masks], batch_size=32)
pred_labels = [np.argmax(pred) for pred in preds[0]]

In [ ]:
for tweet, pred in zip(tweet_dict, pred_labels):
  tweet['label'] = pred